In [1]:
import os
import pandas as pd
import numpy as np
import csv

from itertools import cycle, product
import argparse
import warnings

from sklearn.preprocessing import normalize
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, RepeatedStratifiedKFold, train_test_split
#from sklearn import cross_validation, metrics
from sklearn.utils import shuffle
from sklearn.decomposition import NMF
from sklearn.exceptions import ConvergenceWarning

import matplotlib
from matplotlib import pylab
import matplotlib.pyplot as plt
from IPython.display import Image
%config InlineBackend.figure_format = 'svg'

# import private scripts
import load_kmer_cnts_jf
import stats_utils_AEB

/pollard/home/abustion/deep_learning_microbiome/scripts/stats_utils_AEB.py:2: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  matplotlib.use('Agg') # this suppresses the console for plotting


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, accuracy_score, f1_score, precision_score, recall_score
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.metrics import precision_recall_curve

import operator

import pickle

import logging

In [2]:
graph_dir = os.environ['HOME'] + '/deep_learning_microbiome/tmp_intermediate_files/'

In [ ]:
kmer_size = None
# number of folds for grid search
cv_gridsearch = None
# number of folds for actual training of the best model
cv_testfolds = None
# number of iterations of cross-validation
n_iter = None
random_state = None

plot_fold = False

# Per-iteration plotting
plot_iter = False
# Overall plotting - aggregate results across both folds and iteration
plot_overall = True

# controls transparency of the CI (Confidence Interval) band around the ROCs as in Pasolli
plot_alpha = 0.2

# factor for calculating band around the overall ROC with per fold ROCs (see Pasolli).
plot_factor = 2.26

class_name = "Disease"

graph_dir = os.environ['HOME'] + '/deep_learning_microbiome/analysis/kmers'


plot_title_size = 12
plot_text_size = 10

dataset_config_iter_fold_results = {}

In [ ]:
model_param_grid = {
    "rf": {'DS': [["Zeller_2014"],["Zeller_2014"]],  'CVT': 10,'N': 20,'M': "rf",'CL': [0, 1],
             'CR': 'gini', 'MD': None, 'MF': 'log2', 'MS': 5, 'NE': 400, 'NJ': 1, 'KS': 6, 'NR': 1, 'SL':0}
}

In [5]:
def plot_precision_recall(precision, recall,  average_precision, f1_score, name ='precision_recall', config = ''):
    fig = pylab.figure()

    plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
    plt.fill_between(recall, precision, step='post', alpha=0.2,
         color='b')

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    pylab.gca().set_position((.1, .7, 0.8, .8))
    add_figtexts_and_save(fig, name, '2-class Precision-Recall curve: AP={0:0.4f}, F1={1:0.4f}'.format(
              average_precision, f1_score), config=config)
    
def plot_confusion_matrix(cm, name = 'confusion_mat', config='', cmap=pylab.cm.Reds):
    """
    This function plots the confusion matrix.
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-selection-plot-confusion-matrix-py
    """
    fig, ax = pylab.subplots(1, 2)
    for sub_plt, conf_mat, title, fmt in zip(ax, [cm, cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]], ['Unnormalized Confusion Matrix', 'Normalized Confusion Matrix'], ['d', '.2f']):
        im = sub_plt.imshow(conf_mat, interpolation='nearest', cmap=cmap)
        sub_plt.set_title(title, size=plot_title_size)
        divider = make_axes_locatable(sub_plt)
        cax1 = divider.append_axes("right", size="5%", pad=0.1)
        #fig.colorbar(im, ax=sub_plt)
        fig.colorbar(im, cax=cax1)
        tick_marks = np.arange(len(cm))
        sub_plt.set_xticks(tick_marks)
        sub_plt.set_yticks(tick_marks)
        sub_plt.set_xticklabels(classes)
        sub_plt.set_yticklabels(classes)
        sub_plt.tick_params(labelsize=plot_text_size, axis='both')
        thresh = 0.8*conf_mat.max()
        for i, j in product(range(conf_mat.shape[0]), range(conf_mat.shape[1])):
            sub_plt.text(j, i, format(conf_mat[i, j], fmt),
                         horizontalalignment="center",
                         color="white" if conf_mat[i, j] > thresh else "black", size=plot_title_size)
        sub_plt.set_ylabel('True Label', size=plot_text_size)
        sub_plt.set_xlabel('Predicted Label', size=plot_text_size)
    pylab.tight_layout()
    pylab.gca().set_position((.1, 10, 0.8, .8))
    add_figtexts_and_save(fig, name , "Confusion matrix for predicting sample's " + config + " status ", y_off=1.3, config=config)
    
def plot_roc_aucs(fpr, tpr, roc_auc, accs, std_down=None, std_up=None, config='', name= '', title='ROC Curves with AUCs/ACCs', 
                  desc="ROC/AUC plots using 5mers", xlabel='False Positive Rate', ylabel='True Positive Rate'):
    fig = pylab.figure()
    lw = 2
    if n_classes > 2:
        pylab.plot(fpr["micro"], tpr["micro"],
                   label='micro-average ROC (AUC = {0:0.4f})'
                   ''.format(roc_auc["micro"]),
                   color='deeppink', linestyle=':', linewidth=4)
        if (std_down is not None) and (std_up is not None):
            pylab.fill_between(fpr['micro'], std_down['micro'], std_up['micro'], color='deeppink', lw=0, alpha=plot_alpha)

        pylab.plot(fpr["macro"], tpr["macro"],
                   label='macro-average ROC (AUC = {0:0.4f})'
                   ''.format(roc_auc["macro"]),
                   color='navy', linestyle=':', linewidth=4)
        if (std_down is not None) and (std_up is not None):
            pylab.fill_between(fpr['macro'], std_down['macro'], std_up['macro'], color='navy', lw=0, alpha=plot_alpha)

        roc_colors = cycle(['green', 'red', 'purple', 'darkorange'])
        for i, color in zip(range(n_classes), roc_colors):
            pylab.plot(fpr[i], tpr[i], color=color, lw=lw,
                       label='ROC of {0} (AUC = {1:0.4f}, acc={2:0.4f})'
                       ''.format(classes[i], roc_auc[i], accs[i]))
            if (std_down is not None) and (std_up is not None):
                pylab.fill_between(fpr[i], std_down[i], std_up[i], color=color, lw=0, alpha=plot_alpha)
    else:
        i = 1
        pylab.plot(fpr[i], tpr[i], color='darkorange', lw=lw,
                   label='ROC of {0} (AUC = {1:0.4f}, acc={2:0.4f})'
                   ''.format(classes[i], roc_auc[i], accs[i]))
        if (std_down is not None) and (std_up is not None):
            pylab.fill_between(fpr[i], std_down[i], std_up[i], color='darkorange', lw=0, alpha=plot_alpha)
    
    pylab.plot([0, 1], [0, 1], 'k--', lw=lw)
    pylab.xlim([0.0, 1.0])
    pylab.ylim([0.0, 1.05])
    pylab.xlabel(xlabel)
    pylab.ylabel(ylabel)
    pylab.title(title, size=plot_title_size)
    pylab.legend(loc="lower right", prop={'size': plot_text_size})
    pylab.gca().set_position((.1, .7, .8, .8))
    add_figtexts_and_save(fig, name + "roc_auc", desc, config=config)
    
def add_figtexts_and_save(fig, name, desc, x_off=0.02, y_off=0.56, step=0.04, config=None):
    filename = graph_dir + '/' + name + '_' + config + '.png'
    pylab.figtext(x_off, y_off, desc)
    pylab.savefig(filename , bbox_inches='tight')
    pylab.close(fig)

In [6]:
#Functions
def class_to_target(cls):
    target = np.zeros((n_classes,))
    target[class_to_ind[cls]] = 1.0
    return target

    
def config_info(dataset_name, model_name, config,  kmer_size, skip_keys=['DS', 'CL']):
    config_info = "DS:" + dataset_name
    for k in config:              
        # skip the specified keys, used for skipping the fold and iteration indices (for aggregating results across them)
        if not k in skip_keys:
            config_info += '_' + k + ':' +str(get_config_val(k, config))
    return config_info

def get_config_val(config_key, config):
    val = config[config_key]
    if type(val) is list:
        val = '-'.join([ str(c) for c in val])
    return val

def get_reverse_complement(kmer):
    kmer_rev = ''
    for c in kmer:
        if c == 'A':
            kmer_rev += 'T'
        elif c == 'T':
            kmer_rev += 'A'
        elif c == 'C':
            kmer_rev += 'G'
        else:
            kmer_rev += 'C'

    return kmer_rev[::-1]
    

def get_feature_importances(clf, kmer_imps):
    print("GETTING FEATURE IMPORTANCES")
    importances = clf.feature_importances_
    #std = np.std([tree.feature_importances_ for tree in clf.estimators_],
    #             axis=0)
    for i in range(len(importances)):
        kmer_imps[i] += importances[i]
    print("FINISHED ADDING IMPORTANCES")
    
def get_lasso_importances(estimator, kmer_imps):
    print("GETTING FEATURE IMPORTANCES")
    importances = estimator.coef_
    for i in range(len(importances)):
        kmer_imps[i] += importances[0][i]
    print("FINISHED ADDING IMPORTANCES")
    
def get_lasso_NMF_importances(estimator, factors):
    print("GETTING FEATURE IMPORTANCES")
    importances = estimator.coef_
    for i in range(len(importances)):
        factors[i] += importances[0][i]
    print("FINISHED ADDING IMPORTANCES")

# Random forest CRC

In [ ]:
data_set = 'Zeller_2014'
kmer_size = 6

In [ ]:
kmers_no_comp = []
print("GENERATING ALL KMERS CAPS")
all_kmers_caps = [''.join(_) for _ in product(['A', 'C', 'G', 'T'], repeat = kmer_size)]
print("GENERATED ALL KMERS CAPS")
for kmer in all_kmers_caps:
    if get_reverse_complement(kmer) not in kmers_no_comp:
        kmers_no_comp.append(kmer)
kmer_imps = np.zeros(len(kmers_no_comp))

print("GENERATED KMERS NO COMP")

In [ ]:
config_string = config_info(data_set[0], learn_type, param_grid, kmer_size)

In [ ]:
data_set = [data_set]
allowed_labels = ['0', '1']
kmer_cnts, accessions, labelz, domain_labels = load_kmer_cnts_jf.load_kmers(kmer_size,
                                                                            data_set,
                                                                            allowed_labels)

print("LOADED DATASET " + str(data_set[0]) + ": " + str(len(kmer_cnts)) + " SAMPLES")
labelz=np.asarray(labelz)
labelz=labelz.astype(np.int)

In [ ]:
data_normalized = normalize(kmer_cnts, axis = 1, norm = 'l1')
data_normalized, labels = shuffle(data_normalized, labelz, random_state=0) 
x = data_normalized
y = labels

In [ ]:
dataset_config_iter_fold_results[dataset] = {}
config_results = {}
model = dataset_model_grid[dataset]
param_grid = model_param_grid[model]
classes = param_grid["CL"]
n_classes = len(classes)
global class_to_ind
class_to_ind = { classes[i]: i for i in range(n_classes)}
print("GETTING DATA")
data_set = param_grid["DS"][0]
kmer_size = param_grid["KS"]

In [ ]:
estimator = RandomForestClassifier(n_estimators=400, max_depth=None, min_samples_split=5, n_jobs=1, max_features='log2')
k_fold = RepeatedStratifiedKFold(n_splits=10, n_repeats=20)

In [ ]:
for train_i, test_i in k_fold.split(x, y):
    x_train, y_train = x[train_i], y[train_i]
    x_test, y_test = x[test_i], y[test_i]
    use_norm = True
    print("KFOLD CROSS")
    
    if use_norm:
        sample_mean = x_train.mean(axis=0)
        sample_std = x_train.std(axis=0)

        # Normalize both training and test samples with the training mean and std
        x_train = (x_train - sample_mean) / sample_std
        # test samples are normalized using only the mean and std of the training samples
        x_test = (x_test - sample_mean) / sample_std
                
    y_train = np.array(y_train)
    y_test = np.array(y_test)
                
    estimator.fit(x_train, y_train)
    y_test_pred= np.array(estimator.predict_proba(x_test))
    print("FIT TO ESTIMATOR")
    
    get_feature_importances(estimator, kmer_imps)
    
    conf_mat = confusion_matrix(y_test, np.argmax(y_test_pred, axis=1), labels=range(n_classes))
    if plot_fold:
        plot_confusion_matrix(conf_mat, config = config_string + "_IT_" + str(i) + "_FO_" + str(kfold))

    # printing the accuracy rates for diagnostics
    print("Total accuracy for " + str(len(y_test_pred)) + " test samples: " +
                      str(np.mean(np.equal(y_test, np.argmax(y_test_pred, axis=1)))))

    for cls in classes:
        idx = []
        for j in range(y_test_pred.shape[0]):
            if y_test[j] == cls:
                idx.append(j)
        if len(idx) == 0:
            continue
        idx = np.array(idx)
        print("Accuracy for total of " + str(len(idx)) + " " + str(cls) + " samples: " +
                          str(np.mean(np.equal(y_test[idx], np.argmax(y_test_pred[idx], axis=1)))))

    # Compute ROC curve and AUC for each class - http://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    acc = dict()

                    
    for j in range(n_classes):
        fpr[j], tpr[j], _ = roc_curve(y_test, y_test_pred[:, j])
        roc_auc[j] = auc(fpr[j], tpr[j])
        # Round float 1.0 to integer 1 and 0.0 to 0 in the target vectors, and 1 for max predicted prob
        # index being this one (i), 0 otherwise
        acc[j] = accuracy_score(np.round(y_test), np.equal(np.argmax(y_test_pred, axis=1), j))
                
    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), np.argmax(y_test_pred, axis = 1).ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
                
    # Then interpolate all ROC curves at these points
    mean_tpr = np.zeros_like(all_fpr)
    for j in range(n_classes):
        mean_tpr += np.interp(all_fpr, fpr[j], tpr[j])

    # Finally average it and compute AUC
    mean_tpr /= n_classes


    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    # Plot all ROC curves
    if plot_fold:
        # plot the ROCs with AUCs
        plot_roc_aucs(fpr, tpr, roc_auc, acc, config = config_string + "_IT:" + str(i) + "_FO:" + str(kfold))

    # calculate the accuracy/f1/precision/recall for this test fold - same way as in Pasolli
    test_true_label_inds = y_test
    test_pred_label_inds = np.argmax(y_test_pred, axis = 1)
    accuracy = accuracy_score(test_true_label_inds, test_pred_label_inds)
    f1 = f1_score(test_true_label_inds, test_pred_label_inds, pos_label=None, average='weighted')
    precision = precision_score(test_true_label_inds, test_pred_label_inds, pos_label=None, average='weighted')
    recall = recall_score(test_true_label_inds, test_pred_label_inds, pos_label=None, average='weighted')

    print(('{}\t{}\t{}\t{}\t{}\t{}\tfold-perf-metrics for ' + config_string).
                format(accuracy, f1, precision, recall, roc_auc[1], roc_auc['macro']))
                # the config info for this exp but no fold/iter indices because we need to aggregate stats over them
        
    config_iter_fold_results = dataset_config_iter_fold_results[dataset]
    if config_string not in config_iter_fold_results:
        config_iter_fold_results[config_string] = []

        # the iteration and fold indices
        # extend the list for the iteration if necessary

    if len(config_iter_fold_results[config_string]) <= i:
        config_iter_fold_results[config_string].append([])

    # extend the list for the fold if necessary
    if len(config_iter_fold_results[config_string][i]) <= kfold:
        config_iter_fold_results[config_string][i].append([])

    config_iter_fold_results[config_string][i][kfold] = [conf_mat, [fpr, tpr, roc_auc], classes, [y_test, y_test_pred], [accuracy, f1, precision, recall, roc_auc[1], roc_auc['macro']], shap_values]
    fold_results = np.array(config_iter_fold_results[config_string][i])
    kfold += 1
    logging.info("Completed fold " + str(kfold) + " of " + str(cv_testfolds) + " for iteration " + str(i) + " of " + str(n_iter))

In [ ]:
print("SORTING FEATURE IMPORTANCES")
imps=kmer_imps
num_features = -1
num_feature_imps = num_features
if (num_feature_imps == -1):
    num_feature_imps = len(imps)
if imps is not None and num_feature_imps > 0:
    indices = np.argsort(imps)[::-1][0:num_feature_imps]
    imps = imps[indices]
    kmers_no_comp = [kmers_no_comp[i] for i in indices]
    file = open(graph_dir + "/feat_imps_rf" + str(data_set) + str(kmer_size) + "mers.txt", "w")
    for i in range(num_feature_imps):
        if imps[i] > 0:
            file.write(kmers_no_comp[i] + "\t" + str(imps[i] / (20 * 10)) + "\n")
    print("END FEATURE IMPORTANCE DUMP")
    file.close()

## External validation

### Training model on Zeller data

In [4]:
data_set = 'Zeller_2014'
kmer_size = 5

data_set = [data_set]
allowed_labels = ['0', '1']
kmer_cnts, accessions, labelz, domain_labels = load_kmer_cnts_jf.load_kmers(kmer_size,
                                                                            data_set,
                                                                            allowed_labels)

print("LOADED DATASET " + str(data_set[0]) + ": " + str(len(kmer_cnts)) + " SAMPLES")
labelz=np.asarray(labelz)
labelz=labelz.astype(np.int)

Zeller_2014
LOADED DATASET Zeller_2014: 121 SAMPLES


In [5]:
kmers_no_comp = []
print("GENERATING ALL KMERS CAPS")
all_kmers_caps = [''.join(_) for _ in product(['A', 'C', 'G', 'T'], repeat = kmer_size)]
print("GENERATED ALL KMERS CAPS")
for kmer in all_kmers_caps:
    if get_reverse_complement(kmer) not in kmers_no_comp:
        kmers_no_comp.append(kmer)
kmer_imps = np.zeros(len(kmers_no_comp))

print("GENERATED KMERS NO COMP")

GENERATING ALL KMERS CAPS
GENERATED ALL KMERS CAPS
GENERATED KMERS NO COMP


In [6]:
data_normalized = normalize(kmer_cnts, axis = 1, norm = 'l1')
data_normalized, labels = shuffle(data_normalized, labelz, random_state=0) 
x = data_normalized
y = labels

In [7]:
estimator = RandomForestClassifier(n_estimators=500, max_depth=None, min_samples_split=2, n_jobs=1, max_features='sqrt')
k_fold = RepeatedStratifiedKFold(n_splits=10, n_repeats=1)

In [8]:
for train_i, test_i in k_fold.split(x, y):
    x_train, y_train = x[train_i], y[train_i]
    x_test, y_test = x[test_i], y[test_i]
    use_norm = True
    print("KFOLD CROSS")
    
    if use_norm:
        sample_mean = x_train.mean(axis=0)
        sample_std = x_train.std(axis=0)

        # Normalize both training and test samples with the training mean and std
        x_train = (x_train - sample_mean) / sample_std
        # test samples are normalized using only the mean and std of the training samples
        x_test = (x_test - sample_mean) / sample_std
                
    y_train = np.array(y_train)
    y_test = np.array(y_test)
                
    estimator.fit(x_train, y_train)
    y_test_pred= np.array(estimator.predict_proba(x_test))
    print("FIT TO ESTIMATOR")
    
    #print(estimator.feature_importances_)
    get_feature_importances(estimator, kmer_imps)
    
    print("Total accuracy for " + str(len(y_test_pred)) + " test samples: " +
                      str(np.mean(np.equal(y_test, np.argmax(y_test_pred, axis=1)))))

KFOLD CROSS
FIT TO ESTIMATOR
GETTING FEATURE IMPORTANCES
FINISHED ADDING IMPORTANCES
Total accuracy for 13 test samples: 0.6153846153846154
KFOLD CROSS
FIT TO ESTIMATOR
GETTING FEATURE IMPORTANCES
FINISHED ADDING IMPORTANCES
Total accuracy for 13 test samples: 0.6923076923076923
KFOLD CROSS
FIT TO ESTIMATOR
GETTING FEATURE IMPORTANCES
FINISHED ADDING IMPORTANCES
Total accuracy for 13 test samples: 0.9230769230769231
KFOLD CROSS
FIT TO ESTIMATOR
GETTING FEATURE IMPORTANCES
FINISHED ADDING IMPORTANCES
Total accuracy for 12 test samples: 0.8333333333333334
KFOLD CROSS
FIT TO ESTIMATOR
GETTING FEATURE IMPORTANCES
FINISHED ADDING IMPORTANCES
Total accuracy for 12 test samples: 0.5833333333333334
KFOLD CROSS
FIT TO ESTIMATOR
GETTING FEATURE IMPORTANCES
FINISHED ADDING IMPORTANCES
Total accuracy for 12 test samples: 0.5
KFOLD CROSS
FIT TO ESTIMATOR
GETTING FEATURE IMPORTANCES
FINISHED ADDING IMPORTANCES
Total accuracy for 12 test samples: 0.75
KFOLD CROSS
FIT TO ESTIMATOR
GETTING FEATURE IMPO

In [ ]:
print("SORTING FEATURE IMPORTANCES")
imps=kmer_imps
num_features = -1
num_feature_imps = num_features
if (num_feature_imps == -1):
    num_feature_imps = len(imps)
if imps is not None and num_feature_imps > 0:
    indices = np.argsort(imps)[::-1][0:num_feature_imps]
    imps = imps[indices]
    kmers_no_comp = [kmers_no_comp[i] for i in indices]
    file = open(graph_dir + "/feat_imps_rf" + str(data_set) + str(kmer_size) + "mers.txt", "w")
    for i in range(num_feature_imps):
        if imps[i] > 0:
            file.write(kmers_no_comp[i] + "\t" + str(imps[i] / (20 * 10)) + "\n")
    print("END FEATURE IMPORTANCE DUMP")
    file.close()

### External validation on Feng data

In [ ]:
data_set = 'Feng'
kmer_size = 8

data_set = [data_set]
allowed_labels = ['0', '1']
kmer_cnts, accessions, labelz, domain_labels = load_kmer_cnts_jf.load_kmers(kmer_size,
                                                                            data_set,
                                                                            allowed_labels)

print("LOADED DATASET " + str(data_set[0]) + ": " + str(len(kmer_cnts)) + " SAMPLES")
labelz=np.asarray(labelz)
labelz=labelz.astype(np.int)

data_normalized = normalize(kmer_cnts, axis = 1, norm = 'l1')
data_normalized, labels = shuffle(data_normalized, labelz, random_state=0) 
x = data_normalized
y = labels

sample_mean = x.mean(axis=0)
sample_std = x.std(axis=0)

# Normalize both training and test samples with the training mean and std
x = (x - sample_mean) / sample_std

In [ ]:
y_score = estimator.predict(x)

In [ ]:
from sklearn.metrics import average_precision_score

average_precision = average_precision_score(y, y_score)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.utils.fixes import signature

precision, recall, _ = precision_recall_curve(y, y_score)

# In matplotlib < 1.5, plt.fill_between does not have a 'step' argument
step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))

In [ ]:
# calculate the fpr and tpr for all thresholds of the classification
probs = estimator.predict_proba(x)
preds = probs[:,1]

from sklearn import metrics 

fpr, tpr, threshold = metrics.roc_curve(y, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

# Lasso w/o NMF

In [3]:
data_set = 'Zeller_2014'
kmer_size = 5

In [7]:
kmers_no_comp = []
print("GENERATING ALL KMERS CAPS")
all_kmers_caps = [''.join(_) for _ in product(['A', 'C', 'G', 'T'], repeat = kmer_size)]
print("GENERATED ALL KMERS CAPS")
for kmer in all_kmers_caps:
    if get_reverse_complement(kmer) not in kmers_no_comp:
        kmers_no_comp.append(kmer)
kmer_imps = np.zeros(len(kmers_no_comp))

print("GENERATED KMERS NO COMP")

GENERATING ALL KMERS CAPS
GENERATED ALL KMERS CAPS
GENERATED KMERS NO COMP


In [8]:
data_set = [data_set]
allowed_labels = ['0', '1']
kmer_cnts, accessions, labelz, domain_labels = load_kmer_cnts_jf.load_kmers(kmer_size,
                                                                            data_set,
                                                                            allowed_labels)

print("LOADED DATASET " + str(data_set[0]) + ": " + str(len(kmer_cnts)) + " SAMPLES")
labelz=np.asarray(labelz)
labelz=labelz.astype(np.int)

Zeller_2014
LOADED DATASET Zeller_2014: 121 SAMPLES


In [9]:
data_normalized = normalize(kmer_cnts, axis = 1, norm = 'l1')
data_normalized, labels = shuffle(data_normalized, labelz, random_state=0) 
x = data_normalized
y = labels

In [10]:
estimator = LogisticRegression(penalty='l1', solver='saga', max_iter=10, n_jobs=1)
k_fold = RepeatedStratifiedKFold(n_splits=10, n_repeats=1)

In [11]:
for train_i, test_i in k_fold.split(x, y):
    x_train, y_train = x[train_i], y[train_i]
    x_test, y_test = x[test_i], y[test_i]
    use_norm = True
    print("KFOLD CROSS")
    
    if use_norm:
        sample_mean = x_train.mean(axis=0)
        sample_std = x_train.std(axis=0)

        # Normalize both training and test samples with the training mean and std
        x_train = (x_train - sample_mean) / sample_std
        # test samples are normalized using only the mean and std of the training samples
        x_test = (x_test - sample_mean) / sample_std
                
    y_train = np.array(y_train)
    y_test = np.array(y_test)
                
    estimator.fit(x_train, y_train)
    y_test_pred= np.array(estimator.predict_proba(x_test))
    print("FIT TO ESTIMATOR")
    
    print("GETTING FEATURE IMPORTANCES")
    importances = estimator.coef_
    for i in range(len(importances.T)):
        kmer_imps[i] += abs(importances[0][i])
    print("FINISHED ADDING IMPORTANCES")
    
    print("Total accuracy for " + str(len(y_test_pred)) + " test samples: " +
                      str(np.mean(np.equal(y_test, np.argmax(y_test_pred, axis=1)))))

KFOLD CROSS
FIT TO ESTIMATOR
GETTING FEATURE IMPORTANCES
FINISHED ADDING IMPORTANCES
Total accuracy for 13 test samples: 0.7692307692307693
KFOLD CROSS
FIT TO ESTIMATOR
GETTING FEATURE IMPORTANCES
FINISHED ADDING IMPORTANCES
Total accuracy for 13 test samples: 0.6923076923076923
KFOLD CROSS
FIT TO ESTIMATOR
GETTING FEATURE IMPORTANCES
FINISHED ADDING IMPORTANCES
Total accuracy for 13 test samples: 0.8461538461538461
KFOLD CROSS
FIT TO ESTIMATOR
GETTING FEATURE IMPORTANCES
FINISHED ADDING IMPORTANCES
Total accuracy for 12 test samples: 0.6666666666666666
KFOLD CROSS
FIT TO ESTIMATOR
GETTING FEATURE IMPORTANCES
FINISHED ADDING IMPORTANCES
Total accuracy for 12 test samples: 0.75
KFOLD CROSS
FIT TO ESTIMATOR
GETTING FEATURE IMPORTANCES
FINISHED ADDING IMPORTANCES
Total accuracy for 12 test samples: 0.75
KFOLD CROSS
FIT TO ESTIMATOR
GETTING FEATURE IMPORTANCES
FINISHED ADDING IMPORTANCES
Total accuracy for 12 test samples: 0.6666666666666666
KFOLD CROSS
FIT TO ESTIMATOR
GETTING FEATURE IMP

/pollard/home/abustion/miniconda3/envs/DL_1118/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/pollard/home/abustion/miniconda3/envs/DL_1118/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/pollard/home/abustion/miniconda3/envs/DL_1118/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/pollard/home/abustion/miniconda3/envs/DL_1118/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/pollard/home/abustion/m

In [12]:
print("SORTING FEATURE IMPORTANCES")
imps=kmer_imps
num_features = -1
num_feature_imps = num_features
if (num_feature_imps == -1):
    num_feature_imps = len(imps)
if imps is not None and num_feature_imps > 0:
    indices = np.argsort(imps)[::-1][0:num_feature_imps]
    imps = imps[indices]
    kmers_no_comp = [kmers_no_comp[i] for i in indices]
    file = open(graph_dir + "/feat_imps_lasso" + str(data_set) + str(kmer_size) + "mers.txt", "w")
    for i in range(num_feature_imps):
        if imps[i] > 0:
            file.write(kmers_no_comp[i] + "\t" + str(imps[i] / (20 * 10)) + "\n")
    print("END FEATURE IMPORTANCE DUMP")
    file.close()

SORTING FEATURE IMPORTANCES
END FEATURE IMPORTANCE DUMP


# Lasso with NMF

In [ ]:
data_set = 'Zeller_2014'
kmer_size = 8

kmers_no_comp=[]
for i in range(20):
    kmers_no_comp.append("Factor" + str(i) +": ")
factor_imps = np.zeros(len(kmers_no_comp))

print("GENERATED KMERS NO COMP")

data_set = [data_set]
allowed_labels = ['0', '1']
kmer_cnts, accessions, labelz, domain_labels = load_kmer_cnts_jf.load_kmers(kmer_size,
                                                                            data_set,
                                                                            allowed_labels)

print("LOADED DATASET " + str(data_set[0]) + ": " + str(len(kmer_cnts)) + " SAMPLES")
labelz=np.asarray(labelz)
labelz=labelz.astype(np.int)

In [ ]:
n=20
data_normalized = normalize(kmer_cnts, axis = 1, norm = 'l1')
data_normalized = stats_utils_AEB.NMF_factor(data_normalized, kmer_size, n_components = int(n), 
                                                     title=(str(data_set) + str(kmer_size) + "mers" 
                                                            + str(n) + "factors"))
data_normalized, labels = shuffle(data_normalized, labelz, random_state=0)
x = data_normalized
y = labels

In [ ]:
estimator = LogisticRegression(penalty='l1', solver='saga', max_iter=1000, n_jobs=4)
k_fold = RepeatedStratifiedKFold(n_splits=10, n_repeats=20)

In [ ]:
for train_i, test_i in k_fold.split(x, y):
    x_train, y_train = x[train_i], y[train_i]
    x_test, y_test = x[test_i], y[test_i]
    use_norm = True
    print("KFOLD CROSS")
    
    if use_norm:
        sample_mean = x_train.mean(axis=0)
        sample_std = x_train.std(axis=0)

        # Normalize both training and test samples with the training mean and std
        x_train = (x_train - sample_mean) / sample_std
        # test samples are normalized using only the mean and std of the training samples
        x_test = (x_test - sample_mean) / sample_std
                
    y_train = np.array(y_train)
    y_test = np.array(y_test)
                
    estimator.fit(x_train, y_train)
    y_test_pred= np.array(estimator.predict_proba(x_test))
    print("FIT TO ESTIMATOR")
    
    print("GETTING FEATURE IMPORTANCES")
    importances = estimator.coef_
    for i in range(len(importances.T)):
        factor_imps[i] += abs(importances[0][i])
    print("FINISHED ADDING IMPORTANCES")
    
    print("Total accuracy for " + str(len(y_test_pred)) + " test samples: " +
                      str(np.mean(np.equal(y_test, np.argmax(y_test_pred, axis=1)))))

In [ ]:
print("SORTING FEATURE IMPORTANCES")
imps=factor_imps
num_features = -1
num_feature_imps = num_features
if (num_feature_imps == -1):
    num_feature_imps = len(imps)
if imps is not None and num_feature_imps > 0:
    indices = np.argsort(imps)[::-1][0:num_feature_imps]
    imps = imps[indices]
    kmers_no_comp = [kmers_no_comp[i] for i in indices]
    file = open(graph_dir + "/feat_imps_lasso_NMF" + str(data_set) + str(kmer_size) + "mers.txt", "w")
    for i in range(num_feature_imps):
        if imps[i] > 0:
            file.write(kmers_no_comp[i] + "\t" + str(imps[i] / (20 * 10)) + "\n")
    print("END FEATURE IMPORTANCE DUMP")
    file.close()

In [ ]:
df_nmf = pd.read_csv("/pollard/home/abustion/deep_learning_microbiome/tmp_intermediate_files/feat_imps_lasso_NMF['Zeller_2014']6mers.txt", sep = '\t', header=None)

In [ ]:
df_nmf.head()

In [ ]:
plt.scatter(y = df_nmf[1], x= df_nmf[0])
plt.ylabel("abs value beta coef")
plt.xlabel("factors")
plt.title("Zeller_2014 6mers with NMF 20")
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig("/pollard/home/abustion/deep_learning_microbiome/analysis/NMF/beta_figures/CRC6mers_most_imp_factors.png", bbox_layout='tight')
plt.show()